# Анализ Транзакционных Данных


Датасет `retail` содержит следующие поля:

- **InvoiceNo**: номер транзакции
- **StockCode**: код товара
- **Description**: описание товара
- **Quantity**: количество единиц товара, добавленных в заказ
- **InvoiceDate**: дата транзакции
- **UnitPrice**: цена за единицу товара
- **CustomerID**: id клиента
- **Country**: страна, где проживает клиент

In [15]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import requests
from urllib.parse import urlencode

In [16]:
def url_ydisk (url):    
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    return download_url    

In [17]:
retail = pd.read_csv(url_ydisk('https://disk.yandex.ru/d/yZClOOiE9bQrdw'), encoding='ISO-8859-1')
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### 1 задание
Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из `retail`.

In [18]:
num_duplicates = retail.duplicated().sum()

In [19]:
print(f'В датасете {num_duplicates} повторяющихся наблюдений.')

В датасете 5268 повторяющихся наблюдений.


In [20]:
# Удалим повторяющиеся наблюдения
retail.drop_duplicates(inplace=True)

### 2 задание
Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 

Сколько всего транзакций отменили пользователи? Считайте, что каждая строка - это отдельная транзакция.

In [23]:
canceled_operations = retail.loc[retail.InvoiceNo.str.startswith('C')]

In [24]:
print(f'{canceled_operations.shape[0]} транзакций отменили пользователи.')

9251 транзакций отменили пользователи.


### 3 задание
Отфильтруйте данные и оставьте в `retail` только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [27]:
filtered_df = retail.loc[retail.Quantity > 0]

In [28]:
print(f'{filtered_df.shape[0]} заказов с количествои больше 0.')

526054 заказов с количествои больше 0.


### 4 задание
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).


In [30]:
orders_germany = retail.query('Country == "Germany"') \
    .groupby('CustomerID') \
    .agg({'InvoiceNo':'nunique'})   # один заказ может содержать несколько строк

In [36]:
N_orders = orders_germany.quantile(q=0.8)[0]  # [0] - избавиться от Series для сравнения
N_orders

9.0

In [43]:
germany_top = orders_germany.query('InvoiceNo > @N_orders').index
germany_top

Float64Index([12471.0, 12472.0, 12474.0, 12476.0, 12481.0, 12500.0, 12569.0,
              12600.0, 12619.0, 12621.0, 12626.0, 12647.0, 12662.0, 12705.0,
              12708.0, 12709.0, 12712.0, 12720.0],
             dtype='float64', name='CustomerID')

### 4 задание
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.



In [44]:
top_retail_germany = retail.loc[retail.CustomerID.isin(germany_top)]
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


### 4 задание
Какой из продуктов добавляли в корзину чаще всего, кроме POST? 


Выведем коды топ-10 самых популярных продуктов, добавляемых в корзину.

In [57]:
top_stocks_germany = top_retail_germany.query('StockCode != "POST"') \
    .groupby('StockCode') \
    .agg({'InvoiceNo':'count'}) \
    .sort_values('InvoiceNo', ascending=False) \
    .head(10).reset_index()
top_stocks_germany

,StockCode,InvoiceNo
0,22326,62
1,22423,55
2,22328,45
3,22467,37
4,22077,34
5,20719,34
6,22556,32
7,22961,30
8,22554,29
9,23245,28


А теперь вывыдем список топ-10 самых популярных продуктов, сравнив StockCode с помощью функции `isin`.

In [71]:
top_products = top_retail_germany.loc[top_retail_germany.StockCode.isin(top_stocks_germany.StockCode)] \
    .agg({'Description':'unique'})
top_products

,Description
0,6 RIBBONS RUSTIC CHARM
1,JAM MAKING SET PRINTED
2,ROUND SNACK BOXES SET OF4 WOODLAND
3,WOODLAND CHARLOTTE BAG
4,PLASTERS IN TIN CIRCUS PARADE
5,REGENCY CAKESTAND 3 TIER
6,ROUND SNACK BOXES SET OF 4 FRUITS
7,GUMBALL COAT RACK
8,PLASTERS IN TIN WOODLAND ANIMALS
9,SET OF 3 REGENCY CAKE TINS


### 5 задание
Создайте колонку Revenue с суммой покупки.

In [77]:
retail['Revenue'] = retail.UnitPrice * retail.Quantity

### 6 задание
Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [78]:
total_revenue_df = retail.groupby('InvoiceNo') \
    .agg({'Revenue':'sum'}) \
    .rename(columns={'Revenue':'TotalRevenue'}) \
    .sort_values('TotalRevenue', ascending=False)

total_revenue_df.head(5)

,TotalRevenue
InvoiceNo,
581483,168469.60
541431,77183.60
574941,52940.94
576365,50653.91
556444,38970.00
